In [ ]:
import pandas as pd
import numpy as np
import mlflow 
from pathlib import Path
from typing import Tuple

In [ ]:
# load processed train and dev data for modelling
def load_data(filepath: str, dataset_type: str) -> pd.DataFrame:
    '''
        Loads processed data from csv source
    
        This function loads processed data from csv file
        that will be used for modelling
        
        Args:
            filepath: path to processed dataset
            dataset_type: whether the dataset is train, dev or test dataframe
            
        Returns:
            pd.DataFrame: returns a dataframe containing processed data
            
        Examples:
            >>> df = load_data('data/processed/train_set.csv')
                df.head()
    '''
    filename = Path(filepath)
    if not filename.exists():
        raise FileNotFoundError(f'File not found! Check filepath and try again later!')
    
    df = pd.read_csv(filename)

    # check that the df is not empty
    if len(df) == 0:
        raise ValueError(f'Dataframe cannot be empty!')
    
    print(f'{dataset_type} dataframe successfully loaded with {df.shape[0]} rows and {df.shape[1]} features')
    
    return df


TRAIN_DATA_PATH = '../data/processed/train_set.csv'
DEV_DATA_PATH = '../data/processed/dev_set.csv'

dev_df = load_data(DEV_DATA_PATH, 'Dev')
train_df = load_data(TRAIN_DATA_PATH, 'Train')



Dev dataframe successfully loaded with 7500 rows and 44 features
Train dataframe successfully loaded with 35000 rows and 44 features


In [ ]:
# perform one last data quality check before modelling
def data_quality_checks(df: pd.DataFrame) -> None:
    '''
        Performs one last data quality check before modelling

        Args:
            df: pandas' dataframe to be validated
        
        Raises:
            ValueError: if any of the validations fail

        Example:
            >>> data_quality_checks(df)
    '''

    missing = df.isnull().sum().sum()
    if missing:
        raise ValueError("The dataset must not contain null values")
    
    n_duplicates = df[df.duplicated()].sum().sum()
    if n_duplicates != 0:
        raise ValueError("The dataset must not contain duplicate rows")

    numerical_cols = df.select_dtypes(include=[np.number]).columns.tolist()
    if len(numerical_cols) != len(df.columns):
        raise ValueError("All columns in the dataset must be numerical")

    print("All validation passed!")

data_quality_checks(train_df)
data_quality_checks(dev_df)

All validation passed!
All validation passed!


In [ ]:
# split the data further into features and target set
def features_target_split(df: pd.DataFrame, target: str = 'Current_Salary') -> Tuple[pd.DataFrame, pd.DataFrame]:
    '''
        Split the dataset further into features and target splits
        
        This function takes the given dataframe and splits it
        into the feature set and target set for modelling purposes
    
        Args:
            df: pd.DataFrame = Provided dataframe
            target: str = Target variable in the given dataframe

        Returns:
            A tuple of the features and target sets

        Examples:
            >>> x, y = features_target_split(df, 'Employee_salary')
                x.head()
                y.head()
    '''
    y = df[target].copy()
    if not isinstance(y, pd.Series):
        raise ValueError('The target variable must be a pandas series')
    
    if len(y) == 0:
        raise ValueError(f'The target variable cannot be empty')
    
    x = df.drop(columns=[target]).copy()
    if not isinstance(x, pd.DataFrame):
        raise ValueError('The feature set must be a pandas dataframe')
    
    if len(x) == 0:
        raise ValueError(f'The target variable cannot be empty')
    
    if len(y) != len(x):
        raise ValueError('The length of the target variable must be equal to the length of the feature set')

    
    return x, y

x_train, y_train = features_target_split(train_df)
x_dev, y_dev = features_target_split(dev_df)

In [ ]:
print('x_train: ', x_train.shape)
print(f'x_dev: {x_dev.shape}')
print(f'y_train: {y_train.shape}')
print(f'y_dev: {y_dev.shape}')

x_train:  (35000, 43)
x_dev: (7500, 43)
y_train: (35000,)
y_dev: (7500,)


In [ ]:
y_train

In [ ]:
from sklearn.model_selection import GridSearchCV, KFold, cross_val_score, learning_curve
from sklearn.linear_model import Ridge
from sklearn.ensemble import RandomForestRegressor
from xgboost import XGBRegressor
from lightgbm import LGBMRegressor
from sklearn.inspection import permutation_importance
from sklearn.metrics import mean_squared_error,mean_absolute_error,root_mean_squared_error,r2_score
import matplotlib.pyplot as plt
import seaborn as sns
import time

In [ ]:
# baseline models - performing modelling with minimal feature engineering
models = {
    'Ridge' : Ridge(random_state=42),
    'RandomForest': RandomForestRegressor(n_estimators=200, max_depth=6, n_jobs=-1, random_state=1),
    'XGBoost': XGBRegressor(),
    'LightGBM': LGBMRegressor(random_state=2)
}

for model_name, model in models.items():
    print(f'Training model: {model_name}...')
    start_time = time.time()
    model.fit(x_train, y_train)

    time_elapsed = time.time() - start_time

    preds_train = model.predict(x_train)
    # preds_dev = model.predict(x_dev)

    results = {
        'Model_name' : model_name,
        'Mean_squared_error' : mean_squared_error(y_train, preds_train),
        'Root_mean_squared_error' : root_mean_squared_error(y_train, preds_train),
        'Mean_absolute_error' : mean_absolute_error(y_train, preds_train)
    }
    
    for name, result in results.items():
        print(f'{name} : {result}')
        
    print('='*50)